# SIRF Viewer - Jupyter Notebook Example

This notebook demonstrates how to use the SIRF viewer package in Jupyter notebooks for interactive viewing of SIRF ImageData and AcquisitionData objects.

In [ ]:
# Import necessary packages
import numpy as np
import sys
import os

# Add the src directory to the path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), "src"))

try:
    import sirf.STIR as sirf

    SIRF_AVAILABLE = True
    print("SIRF package is available")
except ImportError:
    SIRF_AVAILABLE = False
    print("Warning: SIRF not available. Using mock data for demonstration.")

from sirf_viewer import (
    SIRFViewer,
    NotebookViewer,
    create_gif_from_data,
    save_view_as_image,
)
from sirf_viewer.utils import print_data_info

## Create Mock Data

For this example, we'll create mock data since we don't have actual SIRF files. In real usage, you would load data from .hv (ImageData) or .hs (AcquisitionData) files.

In [ ]:
def create_mock_image_data():
    """Create mock ImageData for demonstration."""

    class MockImageData:
        def __init__(self, shape):
            self.shape = shape
            # Create more realistic medical imaging data
            self._data = create_realistic_image_data(shape)

        def asarray(self):
            return self._data

        @property
        def __class__(self):
            return type("ImageData", (), {"__name__": "ImageData"})

    return MockImageData((20, 128, 128))


def create_mock_acquisition_data():
    """Create mock AcquisitionData for demonstration."""

    class MockAcquisitionData:
        def __init__(self, shape):
            self.shape = shape
            # Create more realistic acquisition data
            self._data = create_realistic_acquisition_data(shape)

        def asarray(self):
            return self._data

        @property
        def __class__(self):
            return type("AcquisitionData", (), {"__name__": "AcquisitionData"})

    return MockAcquisitionData((8, 16, 64, 64))


def create_realistic_image_data(shape):
    """Create realistic medical image data."""
    z, y, x = shape
    data = np.zeros((z, y, x))

    # Create some structures
    for i in range(z):
        # Add some noise
        noise = np.random.normal(0, 50, (y, x))

        # Add some structures
        center_y, center_x = y // 2, x // 2

        # Create circular structure
        Y, X = np.ogrid[:y, :x]
        dist_from_center = np.sqrt((Y - center_y) ** 2 + (X - center_x) ** 2)

        # Add circular feature
        radius = 20 + i * 2
        circle = (dist_from_center <= radius).astype(float)

        # Add some variation along z
        intensity = 500 + i * 20

        data[i] = noise + circle * intensity

    return data


def create_realistic_acquisition_data(shape):
    """Create realistic acquisition data."""
    tof, views, radial, axial = shape
    data = np.zeros((tof, views, radial, axial))

    for t in range(tof):
        for v in range(views):
            # Create sinogram-like data
            noise = np.random.normal(0, 10, (radial, axial))

            # Add some signal
            center_r, center_a = radial // 2, axial // 2

            R, A = np.ogrid[:radial, :axial]

            # Create some structure
            signal = np.exp(-((R - center_r) ** 2 + (A - center_a) ** 2) / 200)

            # Add view-dependent variation
            angle = v * np.pi / views
            signal *= 1 + 0.5 * np.sin(angle)

            # Add ToF-dependent variation
            tof_factor = np.exp(-t / 3)
            signal *= tof_factor

            data[t, v] = noise + signal * 100

    return data


# Create mock data
image_data = create_mock_image_data()
acq_data = create_mock_acquisition_data()

print("Created mock data:")
print(f"ImageData shape: {image_data.shape}")
print(f"AcquisitionData shape: {acq_data.shape}")

## Basic Data Information

Let's examine the data we created:

In [ ]:
print("=== ImageData Information ===")
print_data_info(image_data)

print("\n=== AcquisitionData Information ===")
print_data_info(acq_data)

## Using SIRFViewer (Matplotlib-based)

First, let's use the basic SIRFViewer which uses matplotlib for display:

In [ ]:
# Create viewer for ImageData
image_viewer = SIRFViewer(image_data, "Example Image Data")

# Display the viewer
# Note: In a real notebook, this would show an interactive plot
# For this example, we'll just save some views
print("Created ImageData viewer")
print(f"Current slice indices: {image_viewer.current_indices}")
print(f"Dimension names: {image_viewer.dimension_names}")

In [ ]:
# Save some different slices
slices_to_save = [5, 10, 15]

for i, slice_idx in enumerate(slices_to_save):
    image_viewer.current_indices[0] = slice_idx
    filename = f"image_slice_{slice_idx:03d}.png"
    save_view_as_image(
        image_data, filename, indices=[slice_idx, 64, 64], colormap="viridis"
    )
    print(f"Saved slice {slice_idx} to {filename}")

In [ ]:
# Create viewer for AcquisitionData
acq_viewer = SIRFViewer(acq_data, "Example Acquisition Data")

print("\nCreated AcquisitionData viewer")
print(f"Current indices: {acq_viewer.current_indices}")
print(f"Dimension names: {acq_viewer.dimension_names}")

# Save some different views
views_to_save = [(2, 4), (4, 8), (6, 12)]

for i, (tof_idx, view_idx) in enumerate(views_to_save):
    acq_viewer.current_indices[0] = tof_idx
    acq_viewer.current_indices[1] = view_idx
    filename = f"acquisition_view_tof{tof_idx}_view{view_idx}.png"
    save_view_as_image(
        acq_data, filename, indices=[tof_idx, view_idx, 32, 32], colormap="plasma"
    )
    print(f"Saved view ToF={tof_idx}, View={view_idx} to {filename}")

## Creating GIF Animations

Let's create animated GIFs from the data:

In [ ]:
# Create GIF from ImageData (animate through z-slices)
print("Creating ImageData animation...")
create_gif_from_data(
    image_data, "image_animation.gif", fps=5, dimensions=[0], colormap="viridis"
)
print("Created: image_animation.gif")

# Create GIF from AcquisitionData (animate through ToF bins)
print("\nCreating AcquisitionData animation...")
create_gif_from_data(
    acq_data, "acquisition_animation.gif", fps=3, dimensions=[0], colormap="plasma"
)
print("Created: acquisition_animation.gif")

## Using NotebookViewer (Interactive Widgets)

Now let's use the NotebookViewer which provides interactive widgets for Jupyter notebooks. Note that this requires ipywidgets to be installed.

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display

    WIDGETS_AVAILABLE = True
    print("ipywidgets is available - interactive widgets will work")
except ImportError:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available - install with: pip install ipywidgets")

if WIDGETS_AVAILABLE:
    # Create notebook viewer for ImageData
    print("\n=== Interactive ImageData Viewer ===")
    nb_image_viewer = NotebookViewer(image_data, width=600, height=400)

    # This would display interactive widgets in a real notebook
    # nb_image_viewer.show()
    print("Notebook viewer created (would show interactive widgets)")

    # Create notebook viewer for AcquisitionData
    print("\n=== Interactive AcquisitionData Viewer ===")
    nb_acq_viewer = NotebookViewer(acq_data, width=600, height=400)

    # This would display interactive widgets in a real notebook
    # nb_acq_viewer.show()
    print("Notebook viewer created (would show interactive widgets)")
else:
    print("\nSkipping interactive widget examples (ipywidgets not available)")

## Advanced Usage Examples

Let's explore some advanced features:

In [ ]:
# Example 1: Using different colormaps
print("=== Colormap Examples ===")
colormaps = ["gray", "viridis", "plasma", "inferno", "magma", "cividis"]

for colormap in colormaps:
    image_viewer.set_colormap(colormap)
    filename = f"image_colormap_{colormap}.png"
    save_view_as_image(image_data, filename, indices=[10, 64, 64], colormap=colormap)
    print(f"Saved {colormap} colormap to {filename}")

In [ ]:
# Example 2: Using window/level settings
print("\n=== Window/Level Examples ===")

# Calculate optimal window/level
from sirf_viewer.utils import get_optimal_window_level

optimal_level, optimal_width = get_optimal_window_level(image_data)
print(f"Optimal window/level: width={optimal_width:.1f}, level={optimal_level:.1f}")

# Try different window/level settings
window_level_settings = [
    (1000, 500),  # Wide window, low level
    (500, 250),  # Medium window, medium level
    (200, 100),  # Narrow window, low level
    (optimal_width, optimal_level),  # Optimal settings
]

for i, (width, level) in enumerate(window_level_settings):
    image_viewer.set_window(level, width)
    filename = f"image_windowlevel_{i:02d}_w{int(width)}_l{int(level)}.png"
    save_view_as_image(image_data, filename, indices=[10, 64, 64], colormap="gray")
    print(f"Saved window/level {width}/{level} to {filename}")

In [ ]:
# Example 3: Creating thumbnails
print("\n=== Thumbnail Creation ===")

from sirf_viewer.utils import create_thumbnail

# Create thumbnails at different sizes
thumbnail_sizes = [(64, 64), (128, 128), (256, 256)]

for size in thumbnail_sizes:
    thumbnail = create_thumbnail(image_data, size=size, indices=[10, 64, 64])
    filename = f"image_thumbnail_{size[0]}x{size[1]}.png"

    # Save thumbnail
    import matplotlib.pyplot as plt

    plt.imsave(filename, thumbnail, cmap="viridis")
    print(f"Saved {size[0]}x{size[1]} thumbnail to {filename}")

## Real Usage Examples

Here's how you would use the viewer with real SIRF data files:

In [ ]:
print("=== Real Usage Examples ===")
print("\nWith real SIRF data files, you would use:")
print()
print("# Load ImageData from .hv file")
print("real_image_data = sirf.ImageData('path/to/your/image.hv')")
print("real_viewer = SIRFViewer(real_image_data)")
print("real_viewer.show()  # Display interactive viewer")
print()
print("# Load AcquisitionData from .hs file")
print("real_acq_data = sirf.AcquisitionData('path/to/your/data.hs')")
print("real_acq_viewer = SIRFViewer(real_acq_data)")
print("real_acq_viewer.show()  # Display interactive viewer")
print()
print("# Create animations")
print("create_gif_from_data(real_image_data, 'real_image.gif', fps=10)")
print("create_gif_from_data(real_acq_data, 'real_acq.gif', fps=5)")
print()
print("# In Jupyter notebook with ipywidgets:")
print("nb_viewer = NotebookViewer(real_image_data)")
print("nb_viewer.show()  # Interactive widgets in notebook")

## Summary

This notebook demonstrated the key features of the SIRF viewer package:

1. **Basic Viewing**: Use `SIRFViewer` for matplotlib-based viewing of both ImageData and AcquisitionData
2. **Interactive Controls**: Sliders for navigating through dimensions, colormap selection, window/level controls
3. **Animation**: Create GIF animations by animating through different dimensions
4. **Export**: Save individual slices or entire animations
5. **Notebook Integration**: Use `NotebookViewer` for interactive widgets in Jupyter notebooks
6. **Advanced Features**: Optimal window/level calculation, thumbnail creation, batch processing

The package is designed to work seamlessly with real SIRF data files (.hv for ImageData, .hs for AcquisitionData) while also providing a flexible interface for custom data processing workflows.